In [30]:
import pandas as pd
from fireagg.database.db import connect

with connect() as commands:
    results = commands.query("""
    SELECT symbol, connector, mid_price, best_bid, best_ask, timestamp, h.update_timestamp, h.fetch_timestamp
    FROM history.symbol_prices_history h
    JOIN realtime.symbols s ON s.id = symbol_id
    """)


data = pd.DataFrame(results)

In [31]:
import plotly.express as px

display(px.histogram(data, x="timestamp"))

data.sort_values('timestamp', inplace=True)
data.to_parquet('connector_latency_v2.parquet', allow_truncated_timestamps=True, coerce_timestamps="us")

# Very slow
# display(px.line(data, x=data.index, y='mid_price', color='connector'))

In [32]:
import datetime
import plotly.express as px

for symbol in data.symbol.unique():
    symbol_data = data[data.symbol == symbol]
    fig = px.line(
        symbol_data,
        x="timestamp",
        y="mid_price",
        color="connector",
        title=symbol,
        line_shape='hv'
    )
    display(fig)

In [5]:
import numpy as np
from filterpy.kalman import KalmanFilter, ExtendedKalmanFilter

data_symbol = data[data.symbol == 'CRV/USDT']
prices_by_connector = pd.pivot_table(data=data_symbol, index='timestamp', columns='connector', values='price')
prices_by_connector = prices_by_connector.ffill().bfill()

predictions = []
f = KalmanFilter(dim_x=1, dim_z=len(prices_by_connector.columns))
f.x = prices_by_connector.iloc[0].mean()
for ts, row in prices_by_connector.iterrows():
    f.predict()
    f.update(row.to_numpy())
    predictions.append(f.x[0][0])


In [6]:
predictions

[0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500